# 一次元CNNによる学習と識別性能評価（TDA）

---
  
引数：raw_rest.csv/raw_tapping.csvがあるディレクトリまでのパス  
  
---
  
入力：raw_rest.csv/raw_tapping.csv

---
  
出力：ACCURACY[loo or k_cv]_TDAautocor_1dCNN.csv　識別性能評価結果一覧  

---  
  
1dCNNを用いて学習し，交差検証法（k-分割交差検証，leave-one-out交差検証）を用いて識別性能評価を行う．  
ベクトル：各ボクセルの時系列データ

In [1]:
print('########### ML_1dCNN_TDAautocor.py program excution ############')

########### ML_1dCNN_TDAautocor.py program excution ############


In [4]:
import pandas as pd
import sys

# matplotlib inline部分は.pyの時にはコメントアウトしないとエラー出る！

import numpy as np
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Flatten
%matplotlib inline
import matplotlib.pyplot as plt
from keras.layers.convolutional import Conv1D, UpSampling1D
from keras.layers.pooling import MaxPooling1D
from keras.utils import np_utils

from sklearn.model_selection import KFold
from sklearn.model_selection import train_test_split

コマンドライン引数でraw_rest.csv/raw_tapping.csvファイルがあるディレクトリまでのパスを取得

In [12]:
#args = sys.argv
#PATH = args[1]

# jupyter notebookのときはここで指定
PATH = '../State-2fe_Active/20181029rn/64ch/RawData/'

# 検証手法
col_name = 'leave-one-out'

# 機械学習するデータ（csvファイルのパラメータ名前）
PM_kizami = ['100', '300']
PM_hole = ['01dim', '012dim']

## ONEdCNN_LOO関数

引数として教師データをX，ラベルをyで受け取る．  
交差検証法の一つleave-one-out交差検証で識別精度評価を行う．

* (1個をテストデータ，残りを教師データにして学習・評価) * すべてのデータ個
* 得られたすべてのデータ個の評価結果（識別率）の平均を求めてパーセントに直す
* 評価結果（識別率）をmain関数に返す

In [20]:
def ONEdCNN_LOO(X, y):

    # 識別率を格納する配列
    LOOscore = np.zeros(len(X))
    
    # ベクトルの長さを格納しておく
    bach_size = X.shape[1]

    
    # 1個をテストデータ，残りを教師データにして学習・評価
    # すべてのデータに対して行う
    for i in range(len(X)):
        
        print('------ ' + str(i) + ' / ' + str(len(X)) + '回 -----')

        # テストデータ
        X_test = X[i]
        y_test = y[i]

        # テストデータとして使用するデータを除いた教師データを作成
        X_train = np.delete(X, i, 0)
        y_train = np.delete(y, i, 0)


        # （データ数, ベクトルの長さ，1）という形にリシェイプする
        X_train = np.reshape(X_train, (-1, bach_size, 1))
        X_test = np.reshape(X_test, (-1, bach_size, 1))
        
        # ダミー変数に変換：何分類するかによって数字を書き換える（rest, Tappingの2分類なので2）
        y_train = np_utils.to_categorical(y_train, 2)
        y_test = np_utils.to_categorical(y_test, 2)


        # 1次元CNNのインスタンスを作成
        # 参考文献（Time series classification via TDA）に従ってパラメータを設定
        # 1st Conv1d : kernel_number = 7, kernel_size = 6
        # 1st MacPooling : kernel_number = 7
        # 2nd Conv1d : kernel_number = 7, kernel_size = 2
        # 2nd MacPooling : kernel_number = 3
        # Flattenで1次元に
        model = Sequential()
        model.add(Conv1D(7, 6, padding='same', input_shape=(bach_size, 1)))
        model.add(Activation('relu'))
        model.add(MaxPooling1D(7, padding='same'))
        model.add(Conv1D(7, 2, padding='same', activation='relu'))
        model.add(MaxPooling1D(3, padding='same'))

        model.add(Flatten())
        
        # 何分類するかによってunitsを書き換える（rest, Tappingの2分類なのでunits=2）
        model.add(Dense(units=2, activation='softmax'))
        model.compile(loss='mse', optimizer='adam', metrics=["accuracy"])

        # 学習
        history = model.fit(X_train, y_train, epochs=100)
        
        # 評価結果（識別率）を格納
        score = model.evaluate(X_test, y_test)
        LOOscore[i] = score[1]
        

        
    # 評価結果（識別率）の平均を求める
    result = LOOscore.mean()
    
    # パーセントに直す
    result = round(result * 100, 1)
    
    print(str(LOOscore) + '\n')

    return result

## ONEdCNN_kCV関数

引数として教師データをX，ラベルをyで受け取る．  
交差検証法の一つk-分割交差検証で識別精度評価を行う．

* 学習
* (k分割し，1グループをテストデータ，残りグループを教師データにして評価) * k
* 得られたk個の評価結果（識別率）の平均を求めてパーセントに直す
* 評価結果（識別率）をmain関数に返す

In [21]:
def ONEdCNN_kCV(X, y):
    
    
    # 識別率を格納する配列
    kCVscore = np.zeros(cv_k)
    
    # ベクトルの長さを格納しておく
    bach_size = X.shape[1]
    
    # 分割数
    kf = KFold(n_splits=cv_k, shuffle=False)
    # 繰り返し回数
    i = 0

    for train_index, eval_index in kf.split(X):
        
        print('------ ' + str(i) + ' / ' + str(cv_k) + '回 -----')
        
        X_train, X_test = X[train_index], X[eval_index]
        y_train, y_test = y[train_index], y[eval_index]

        # （データ数, ベクトルの長さ，1）という形にリシェイプする
        X_train = np.reshape(X_train, (-1, bach_size, 1))
        X_test = np.reshape(X_test, (-1, bach_size, 1))

        # ダミー変数に変換：何分類するかによって数字を書き換える（rest, Tappingの2分類なので2）
        y_train = np_utils.to_categorical(y_train, 2)
        y_test = np_utils.to_categorical(y_test, 2)


        # 1次元CNNのインスタンスを作成
        # 参考文献（Time series classification via TDA）に従ってパラメータを設定
        # 1st Conv1d : kernel_number = 7, kernel_size = 6
        # 1st MacPooling : kernel_number = 7
        # 2nd Conv1d : kernel_number = 7, kernel_size = 2
        # 2nd MacPooling : kernel_number = 3
        # Flattenで1次元に
        model = Sequential()
        model.add(Conv1D(7, 6, padding='same', input_shape=(bach_size, 1)))
        model.add(Activation('relu'))
        model.add(MaxPooling1D(7, padding='same'))
        model.add(Conv1D(7, 2, padding='same', activation='relu'))
        model.add(MaxPooling1D(3, padding='same'))

        model.add(Flatten())
        
        # 何分類するかによってunitsを書き換える（rest, Tappingの2分類なのでunits=2）
        model.add(Dense(units=2, activation='softmax'))
        model.compile(loss='mse', optimizer='adam', metrics=["accuracy"])

        # 学習
        history = model.fit(X_train, y_train, epochs=100)
        
        # 評価結果（識別率）を格納
        score = model.evaluate(X_test, y_test)
        kCVscore[i] = score[1]
        
        i = i + 1
        

    # 評価結果（識別率）の平均を求める
    result = kCVscore.mean()
    
    # パーセントに直す
    result = round(result * 100, 1)
    
    print('k = ' + str(cv_k) + '：' + str(kCVscore))

    return result

## TrainingData関数 
引数として読み込みたいタスクごとのファイル名をfile_rest/file_tappingで受け取る． 
* 機械学習にかけれるようにデータのベクトル化とラベルを作成
* ベクトル化したデータとラベルをmain関数に返す

In [22]:
def TrainingData(file_rest, file_tapping):
    
    # 読み込みたいファイルのパス
    PATH_rest = PATH + file_rest
    PATH_tapping = PATH + file_tapping

    # csvファイル読み込み（転地することでベクトルを時系列データとする）
    rest = pd.read_csv(PATH_rest, header = 0)    
    tapping = pd.read_csv(PATH_tapping, header = 0)

    # 各タスクのデータを縦結合
    all_data = pd.concat([rest, tapping], axis = 0)

    # ベクトル化
    X = all_data.as_matrix()

    # ラベル作成 rock = 0, scissor = 1, paper = 2
    label_rest = np.zeros(len(rest.index))
    label_tapping = np.ones(len(tapping.index))

    y = np.r_[label_rest, label_tapping]

    
    return X, y

## main関数

In [23]:
if __name__ == '__main__':
    
        
    for kizami in range(len(PM_kizami)):
        
        for hole in range(len(PM_hole)):
            
            PM_dataName = PM_hole[hole] + PM_kizami[kizami]

            # 機械学習するデータ（提案手法でベクトル化したcsvファイル）
            ML_restData = 'TDAvec_autocor_rest_' + PM_dataName + '.csv'
            ML_tapData = 'TDAvec_autocor_tapping_' + PM_dataName + '.csv'
            
            
            # データとラベルの作成
            data, labels = TrainingData(ML_restData, ML_tapData)

            
            # 学習とleave-one-out交差検証法
    
            print('leave-one-out Cross-Validation : ' + PM_dataName) 

            result_loo = ONEdCNN_LOO(data, labels)

            # データフレーム化
            result_loo = pd.DataFrame({col_name:[result_loo]}, index = ['TDA(' + PM_dataName + ') + 1dCNN'])
            print(result_loo)
            
            # csv書き出し
            PATH_RESULT = PATH + 'ACCURACY[loo]_TDAautocor' + PM_dataName + '_1dCNN.csv'
            result_loo.to_csv(PATH_RESULT, index = True)

leave-one-out Cross-Validation : 01dim100
------ 0 / 14回 -----
Epoch 1/100
13/13 [==============================] - 0s 13ms/step - loss: 0.2197 - acc: 0.6154
Epoch 2/100
13/13 [==============================] - 0s 335us/step - loss: 0.2167 - acc: 0.6923
Epoch 3/100
13/13 [==============================] - 0s 347us/step - loss: 0.2160 - acc: 0.6154
Epoch 4/100
13/13 [==============================] - 0s 408us/step - loss: 0.2134 - acc: 0.6154
Epoch 5/100
13/13 [==============================] - 0s 405us/step - loss: 0.2111 - acc: 0.6923
Epoch 6/100
13/13 [==============================] - 0s 403us/step - loss: 0.2093 - acc: 0.6923
Epoch 7/100
13/13 [==============================] - 0s 439us/step - loss: 0.2078 - acc: 0.6923
Epoch 8/100
13/13 [==============================] - 0s 704us/step - loss: 0.2059 - acc: 0.7692
Epoch 9/100
13/13 [==============================] - 0s 404us/step - loss: 0.2039 - acc: 0.7692
Epoch 10/100
13/13 [==============================] - 0s 474us/step - loss

Epoch 85/100
13/13 [==============================] - 0s 438us/step - loss: 0.0910 - acc: 0.9231
Epoch 86/100
13/13 [==============================] - 0s 419us/step - loss: 0.0896 - acc: 0.9231
Epoch 87/100
13/13 [==============================] - 0s 432us/step - loss: 0.0880 - acc: 0.9231
Epoch 88/100
13/13 [==============================] - 0s 311us/step - loss: 0.0866 - acc: 0.9231
Epoch 89/100
13/13 [==============================] - 0s 411us/step - loss: 0.0852 - acc: 0.9231
Epoch 90/100
13/13 [==============================] - 0s 447us/step - loss: 0.0838 - acc: 0.9231
Epoch 91/100
13/13 [==============================] - 0s 452us/step - loss: 0.0824 - acc: 0.9231
Epoch 92/100
13/13 [==============================] - 0s 505us/step - loss: 0.0809 - acc: 0.9231
Epoch 93/100
13/13 [==============================] - 0s 378us/step - loss: 0.0796 - acc: 0.9231
Epoch 94/100
13/13 [==============================] - 0s 362us/step - loss: 0.0783 - acc: 0.9231
Epoch 95/100
13/13 [==========

Epoch 69/100
13/13 [==============================] - 0s 367us/step - loss: 0.1932 - acc: 0.6923
Epoch 70/100
13/13 [==============================] - 0s 518us/step - loss: 0.1899 - acc: 0.7692
Epoch 71/100
13/13 [==============================] - 0s 368us/step - loss: 0.1870 - acc: 0.6923
Epoch 72/100
13/13 [==============================] - 0s 464us/step - loss: 0.1847 - acc: 0.6154
Epoch 73/100
13/13 [==============================] - 0s 423us/step - loss: 0.1835 - acc: 0.6923
Epoch 74/100
13/13 [==============================] - 0s 439us/step - loss: 0.1826 - acc: 0.6923
Epoch 75/100
13/13 [==============================] - 0s 414us/step - loss: 0.1819 - acc: 0.6923
Epoch 76/100
13/13 [==============================] - 0s 487us/step - loss: 0.1810 - acc: 0.6923
Epoch 77/100
13/13 [==============================] - 0s 510us/step - loss: 0.1797 - acc: 0.6923
Epoch 78/100
13/13 [==============================] - 0s 450us/step - loss: 0.1782 - acc: 0.6923
Epoch 79/100
13/13 [==========

Epoch 53/100
13/13 [==============================] - 0s 387us/step - loss: 0.2517 - acc: 0.4615
Epoch 54/100
13/13 [==============================] - 0s 406us/step - loss: 0.2502 - acc: 0.4615
Epoch 55/100
13/13 [==============================] - 0s 419us/step - loss: 0.2483 - acc: 0.4615
Epoch 56/100
13/13 [==============================] - 0s 415us/step - loss: 0.2465 - acc: 0.4615
Epoch 57/100
13/13 [==============================] - 0s 471us/step - loss: 0.2450 - acc: 0.4615
Epoch 58/100
13/13 [==============================] - 0s 480us/step - loss: 0.2436 - acc: 0.4615
Epoch 59/100
13/13 [==============================] - 0s 540us/step - loss: 0.2423 - acc: 0.4615
Epoch 60/100
13/13 [==============================] - 0s 522us/step - loss: 0.2409 - acc: 0.4615
Epoch 61/100
13/13 [==============================] - 0s 388us/step - loss: 0.2394 - acc: 0.4615
Epoch 62/100
13/13 [==============================] - 0s 515us/step - loss: 0.2379 - acc: 0.4615
Epoch 63/100
13/13 [==========

Epoch 37/100
13/13 [==============================] - 0s 420us/step - loss: 0.1920 - acc: 0.6154
Epoch 38/100
13/13 [==============================] - 0s 738us/step - loss: 0.1902 - acc: 0.6154
Epoch 39/100
13/13 [==============================] - 0s 473us/step - loss: 0.1883 - acc: 0.6154
Epoch 40/100
13/13 [==============================] - 0s 437us/step - loss: 0.1865 - acc: 0.6923
Epoch 41/100
13/13 [==============================] - 0s 417us/step - loss: 0.1862 - acc: 0.7692
Epoch 42/100
13/13 [==============================] - 0s 379us/step - loss: 0.1845 - acc: 0.7692
Epoch 43/100
13/13 [==============================] - 0s 525us/step - loss: 0.1816 - acc: 0.7692
Epoch 44/100
13/13 [==============================] - 0s 398us/step - loss: 0.1802 - acc: 0.7692
Epoch 45/100
13/13 [==============================] - 0s 382us/step - loss: 0.1789 - acc: 0.7692
Epoch 46/100
13/13 [==============================] - 0s 462us/step - loss: 0.1775 - acc: 0.8462
Epoch 47/100
13/13 [==========

...

Epoch 12/100
13/13 [==============================] - 0s 573us/step - loss: 0.2153 - acc: 0.7692
Epoch 13/100
13/13 [==============================] - 0s 612us/step - loss: 0.2129 - acc: 0.7692
Epoch 14/100
13/13 [==============================] - 0s 649us/step - loss: 0.2101 - acc: 0.7692
Epoch 15/100
13/13 [==============================] - 0s 590us/step - loss: 0.2075 - acc: 0.6923
Epoch 16/100
13/13 [==============================] - 0s 594us/step - loss: 0.2058 - acc: 0.6923
Epoch 17/100
13/13 [==============================] - 0s 673us/step - loss: 0.2033 - acc: 0.7692
Epoch 18/100
13/13 [==============================] - 0s 678us/step - loss: 0.1999 - acc: 0.8462
Epoch 19/100
13/13 [==============================] - 0s 818us/step - loss: 0.1973 - acc: 0.8462
Epoch 20/100
13/13 [==============================] - 0s 597us/step - loss: 0.1947 - acc: 0.8462
Epoch 21/100
13/13 [==============================] - 0s 576us/step - loss: 0.1924 - acc: 0.9231
Epoch 22/100
13/13 [==========

Epoch 97/100
13/13 [==============================] - 0s 655us/step - loss: 0.0378 - acc: 1.0000
Epoch 98/100
13/13 [==============================] - 0s 700us/step - loss: 0.0368 - acc: 1.0000
Epoch 99/100
13/13 [==============================] - 0s 681us/step - loss: 0.0359 - acc: 1.0000
Epoch 100/100
1/1 [==============================] - 1s 979ms/step
------ 11 / 14回 -----
Epoch 1/100
13/13 [==============================] - 2s 147ms/step - loss: 0.4810 - acc: 0.4615
Epoch 2/100
13/13 [==============================] - 0s 659us/step - loss: 0.4706 - acc: 0.4615
Epoch 3/100
13/13 [==============================] - 0s 593us/step - loss: 0.4571 - acc: 0.4615
Epoch 4/100
13/13 [==============================] - 0s 2ms/step - loss: 0.4392 - acc: 0.4615
Epoch 5/100
13/13 [==============================] - 0s 932us/step - loss: 0.4195 - acc: 0.4615
Epoch 6/100
13/13 [==============================] - 0s 740us/step - loss: 0.3982 - acc: 0.4615
Epoch 7/100
13/13 [===========================

Epoch 81/100
13/13 [==============================] - 0s 625us/step - loss: 0.1337 - acc: 0.9231
Epoch 82/100
13/13 [==============================] - 0s 661us/step - loss: 0.1314 - acc: 0.9231
Epoch 83/100
13/13 [==============================] - 0s 687us/step - loss: 0.1290 - acc: 0.9231
Epoch 84/100
13/13 [==============================] - 0s 639us/step - loss: 0.1269 - acc: 0.9231
Epoch 85/100
13/13 [==============================] - 0s 662us/step - loss: 0.1247 - acc: 0.9231
Epoch 86/100
13/13 [==============================] - 0s 612us/step - loss: 0.1224 - acc: 0.9231
Epoch 87/100
13/13 [==============================] - 0s 655us/step - loss: 0.1202 - acc: 0.9231
Epoch 88/100
13/13 [==============================] - 0s 635us/step - loss: 0.1180 - acc: 0.9231
Epoch 89/100
13/13 [==============================] - 0s 664us/step - loss: 0.1158 - acc: 0.9231
Epoch 90/100
13/13 [==============================] - 0s 669us/step - loss: 0.1137 - acc: 0.9231
Epoch 91/100
13/13 [==========

Epoch 65/100
13/13 [==============================] - 0s 738us/step - loss: 0.1207 - acc: 0.9231
Epoch 66/100
13/13 [==============================] - 0s 630us/step - loss: 0.1185 - acc: 0.9231
Epoch 67/100
13/13 [==============================] - 0s 689us/step - loss: 0.1163 - acc: 0.9231
Epoch 68/100
13/13 [==============================] - 0s 658us/step - loss: 0.1143 - acc: 0.9231
Epoch 69/100
13/13 [==============================] - 0s 673us/step - loss: 0.1124 - acc: 0.9231
Epoch 70/100
13/13 [==============================] - 0s 603us/step - loss: 0.1105 - acc: 0.9231
Epoch 71/100
13/13 [==============================] - 0s 654us/step - loss: 0.1086 - acc: 0.9231
Epoch 72/100
13/13 [==============================] - 0s 744us/step - loss: 0.1066 - acc: 0.9231
Epoch 73/100
13/13 [==============================] - 0s 665us/step - loss: 0.1046 - acc: 0.9231
Epoch 74/100
13/13 [==============================] - 0s 626us/step - loss: 0.1028 - acc: 0.9231
Epoch 75/100
13/13 [==========

Epoch 49/100
13/13 [==============================] - 0s 680us/step - loss: 0.1382 - acc: 0.8462
Epoch 50/100
13/13 [==============================] - 0s 684us/step - loss: 0.1364 - acc: 0.8462
Epoch 51/100
13/13 [==============================] - 0s 760us/step - loss: 0.1346 - acc: 0.8462
Epoch 52/100
13/13 [==============================] - 0s 679us/step - loss: 0.1328 - acc: 0.8462
Epoch 53/100
13/13 [==============================] - 0s 753us/step - loss: 0.1310 - acc: 0.8462
Epoch 54/100
13/13 [==============================] - 0s 689us/step - loss: 0.1293 - acc: 0.8462
Epoch 55/100
13/13 [==============================] - 0s 749us/step - loss: 0.1276 - acc: 0.8462
Epoch 56/100
13/13 [==============================] - 0s 675us/step - loss: 0.1259 - acc: 0.8462
Epoch 57/100
13/13 [==============================] - 0s 694us/step - loss: 0.1243 - acc: 0.8462
Epoch 58/100
13/13 [==============================] - 0s 703us/step - loss: 0.1227 - acc: 0.8462
Epoch 59/100
13/13 [==========